# Introduction

Measuring the performance of a model is a very critical step in the process of building _machine learning_ models. More specifically, the ultimate goal would be building a model that generalizes well. To better approximate the performance of a model in a real world setting, we need to understand different validation strategies with both their advantages and drawbacks.

# Validation Strategies

The whole idea behind validation set is to have dataset that best approximate the model score when tested on unseen data. Therefore, this will help us avoid overfitting. Also, the validation data should have the same distribution as the test set. In some cases, training data may have different distribution than the test set.
> Overfitting means that the validation error starts increasing (change direction from decreasing) because it starts capturing noise and patterns that are specific to the training data and are not present in the test data. Overfitting doesn't mean that training error is less than validation error because almost always training error is less than validation error.

We will cover the most common strategies used in practice and this is by no means an exhaustive list of all validation strategies. 

## Holdout (Train/Validation/Test)

With this method, we divide the dataset randomly (or based on time for time-sensitive dataset) into three datasets: training, validation, and test datasets. The test dataset __must not be used__ until the modeling process is done. Most common splits are 60/20/20, 70/15/15, and 80/10/10. However, this all depends on the size of the dataset. For example, if the dataset has 10 million samples, then 98/1/1 is good.

The process works as follows:
1. Split the data into train, validation, and test datasets.
2. Fit the model on the training data.
3. Evaluate the model on the validation dataset.
4. When modeling process is done:
    1. Concatenate both training and validation datasets.
    2. Train the final model on the concatenated dataset.
    3. Evaluate the final model on the test dataset and report the metrics.

<p align="center">
<img src="../images/holdout.png" height="300px" width="400px">
    <caption><center><b>Figure 1:</b> Visual summary of validation holdout method.<a href="https://arxiv.org/pdf/1811.12808v2.pdf"> Source</a></center></caption>
</p>

The two main disadvantage of the holdout approach are:
1. Different splits produce different test error rates (high variance). That is because it is based on which observations were on training and which were on test set.
2. Different sizes of test sets have different bias/variance trade-offs. For example, bigger test sets means the model will be trained on fewer number of samples. This would overestimate the test error rate (high bias). However, smaller test sets means the model will have fewer samples to be tested on. As a result, the model would underestimate the test error rate (high variance).

Even with the drawbacks of the this strategy, it is still useful especially in large datasets and computationally intensive models which is the case in computer vision and NLP.

## Repeated Holdout

To overcome the high volatility of _holdout_ method and get more robust estimate of the test error rate, we can use repeated holdout method _k_ times. That is:
1. Split the dataset as we did in the holdout method into training and test sets.
2. Fit the model on the training dataset.
3. Evaluate the model on the test dataset.
4. Repeat the above 3 steps _k_ times and then take the average of the test error rates.

<p align="center">
<img src="../images/repeated-holdout.png" height="300px" width="400px">
<caption><center><b>Figure 2:</b> Visual summary of validation repeated holdout method.<a href="https://arxiv.org/pdf/1811.12808v2.pdf"> Source</a></center></caption>
</p>

This will give us a better estimate of the test error rate. Note that we can use this method in two way:
1. Either split the dataset into training and test datasets and use this method on the training dataset and leave test dataset to the end.
2. Or Use this method as is and don't have a separate test dataset.

## LOOCV

_Leave one out cross validation_ (LOOCV) is very simple. On each round, we train on $n - 1$ samples and test on the sample was left out. We keep doing this until all samples were left out once. When done, take the average of the test error rate from all samples.

<p align="center">
<img src="../images/loocv.png" height="300px" width="400px">
<caption><center><b>Figure 3:</b> Visual summary of LOOCV method.<a href="https://arxiv.org/pdf/1811.12808v2.pdf"> Source</a></center></caption>
</p>

The main advantage of this method is that it has an unbiased estimate of the test error. However, it has high variance due to the fact that training data in each round is almost identical which results in a correlated test error rates. Additionally, it is very expensive computationally especially when we have complex models and/or medium-to-large datasets. As a result, this is typically used when we have very small datasets.

## K-fold Cross Validation (CV)

_K-fold CV_ is a special case of cross validation which means each sample has the probability of being tested at least once. With k-fold CV:
1. We split the data randomly into training and test sets.
2. Fit the model on the training data (training folds).
3. Evaluate the model on test data (test fold).
4. Repeat 1-3 steps __K__ times.
5. Take the average of all the test folds error rates.

<p align="center">
<img src="../images/cv.png" height="300px" width="400px">
<caption><center><b>Figure 4:</b> Visual summary of K-fold CV method.</center></caption>
</p>

As _k_ increases, this would reduce bias and increase the variance of the test error estimates. In the extreme case when $k=n$, then we would end up with _LOOCV_. Typical values of _k_ are 5 or 10 because they don't suffer from excessive bias or variance.

Note that even though _repeated holdout_ and _k-fold cv_ look almost the same, there is a major difference in that each sample is guaranteed to be in the test fold __only once__ in _k-fold cv_; however, some samples may never appear in the test fold and some may appear more than once under _repeated holdout_ strategy.

## Nested K-fold CV

This strategy is useful especially when we have a small dataset that can't reserve an independent test dataset. This method has been shown to reduce bias compared to _k-fold cv_. 

There would be two loops in this strategy: inner loop and outer loop. Here is how it works:
1. Split the dataset into _k-folds_ where one fold will be the test set and the rest $k-1$ will be the training set. This loop is typically used to estimate the test error for the best model chosen from the inner loop.
2. For each training fold:
    1. Split into _k-folds_. This step is typically used for the model selection such as hperparameter tuning.

<p align="center">
<img src="../images/nested-cv.png" height="400px" width="500px">
<caption><center><b>Figure 5:</b> Visual summary of Nested K-fold CV method.<a href="https://arxiv.org/pdf/1811.12808v2.pdf">Source</a></center></caption>
</p>

## Bootstrapping

_Bootstrapping_ is a resampling method that aims to estimate the sampling distribution. In the modeling setting, we are interested in estimating the sampling distribution of the test error rate. This method will give us the uncertainty and confidence intervals of the test error rate so we would have a better idea about the generalization of our model and what to expect in a production setting. 

The method works as follows:
1. We define parameter _b_ which determines how many times to draw samples from the population (dataset).
2. We draw a random sample of size _n_ (size of the original dataset) from the original dataset with replacement. This is different from repeated holdout since each sample can be drawn more than once. The probability of a sample being included is 63.2%.
3. Fit the model on the drawn sample.
4. Compute the error rate.
5. Repeat steps 2-4.

When done, we would end up with _b_ estimates of the test error rate which would be a sampling distribution for our estimates. Therefore, we can get the uncertainty and confidence intervals as well.

However, this approach suffers from the fact that the error rate on each drawn sample is biased because we are training and testing the model on the same dataset. Therefore, an improved version of this method is called _leave-one-out bootstrap_. This means that we calculate the test error rate on each bootstrap step on the samples that were not drawn (not included in the drawn sample). This way, we can overcome the bias issue.

<p align="center">
<img src="../images/loo-bootstrap.png" height="400px" width="500px">
<caption><center><b>Figure 6:</b> Visual summary of Leave-one-out Bootstrap method.<a href="https://arxiv.org/pdf/1811.12808v2.pdf">Source</a></center></caption>
</p>

# Conclusion

Having a good validation strategy help us evaluate the quality of a model and get a better sense of its generalization error in the real world setting. This will guide us to select the model that would potentially perform the best on unseen data by avoiding to fall into overfitting trap where the model performs greatly on our training dataset and performs badly in production. We covered the most common validation strategies and some of their shortcomings. One important aspect that we did not cover here is how to make sure that the validation dataset is representative of the independent test dataset (or what is expected to see in production) and how to avoid overfitting to the validation dataset.

Below are important points to remember when selecting train/validation datasets:
- If the dataset is imbalanced, make sure to use stratified version of validation strategies so that the target distribution of both train and validation datasets is the same.
- Make train/validation split as similar as possible to train/test splits.
- For time-dependent data, validation should be in the future.
- When splitting the data, the underlying assumption is that rows are independent. If that is not the case, we need to make sure that rows that are related don't be split included in both training and validation/test datasets.

# References

\[Efron, 1981\] Efron, B. (1981). Nonparametric standard errors and confidence intervals. _Canadian
Journal of Statistics_ , 9(2):139–158.

\[Efron, 1983\] Efron, B. (1983). Estimating the error rate of a prediction rule: improvement on
cross-validation. _Journal of the American Statistical Association_ , 78(382):316–331.

\[Efron, 1992\] Efron, B. (1992). Bootstrap methods: another look at the Jackknife. In Breakthroughs
in Statistics, pages 569–593. Springer.

\[Efron and Tibshirani, 1997\] Efron, B. and Tibshirani, R. (1997). Improvements on cross-validation:
the .632+ bootstrap method. Journal of the American Statistical Association, 92(438):548–560.

\[Efron and Tibshirani, 1994\] Efron, B. and Tibshirani, R. J. (1994). An Introduction to the Bootstrap.
CRC press

[Raschka, 2018] Raschka, S. (2018). Model Evaluation, Model Selection, and Algorithm Selection in Machine Learning. _arXiv:1811.12808v2_.